<a href="https://colab.research.google.com/github/kalyani234/AI_Agents_Projects/blob/CrewAI/CrewAI_Sales_Outreach_Multi_Agent_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guida

In [57]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [58]:
import os

os.makedirs("instructions", exist_ok=True)

with open("instructions/product_info.txt", "w") as f:
    f.write(
        "DeepLearningAI creates high-quality AI education content, "
        "courses, newsletters, and hands-on learning programs."
    )

with open("instructions/market_notes.md", "w") as f:
    f.write(
        "# Market Notes\n"
        "- AI education demand is rising.\n"
        "- Organizations need practical AI upskilling.\n"
        "- DeepLearningAI is a leader in the ecosystem."
    )

with open("instructions/sales_strategy.txt", "w") as f:
    f.write(
        "When contacting executives, personalize with:\n"
        "- their recent announcements,\n"
        "- their education initiatives,\n"
        "- their AI community efforts."
    )

print("Instruction files created!")


Instruction files created!


In [59]:
from crewai import Agent, Task, Crew
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool
from google.colab import userdata
import os


In [60]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()


In [61]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("open_api")
os.environ["SERPER_API_KEY"] = userdata.get("serpi_api")

# required by CrewAI v0.28
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"


Creating Agents

In [62]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [63]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

Creating Tools

In [64]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [65]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

Custom Tool

In [66]:
from crewai_tools import BaseTool

In [67]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [68]:
sentiment_analysis_tool = SentimentAnalysisTool()

Creating Tasks
- The Lead Profiling Task is using crewAI Tools.

In [69]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [70]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

Creating the Crew

In [71]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	memory=True
)

Running the Crew

In [72]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather information about DeepLearningAI, focusing on their company background, key personnel, recent milestones, and potential needs that align with our offerings. To do this effectively, I will start by searching the internet for relevant information about DeepLearningAI.

Action: Search the internet  
Action Input: {"search_query": "DeepLearningAI company profile, key decision-makers, recent d

# Display the final result as Markdown.

In [74]:
from IPython.display import Markdown
Markdown(result)

### Email Draft 1: Introduction to Our Solutions

**Subject:** Empowering DeepLearning.AI's Vision with Tailored Solutions

Dear Andrew,

I hope this message finds you well. I wanted to take a moment to congratulate you on the successful launch of your new "Agentic AI" course. It's inspiring to see DeepLearning.AI continue to lead the charge in democratizing AI education and empowering individuals to harness the power of AI.

At CrewAI, we share your commitment to advancing education through innovative solutions. With the increasing demand for AI skills, we understand the importance of providing your learners with engaging, high-quality educational experiences. Our customizable learning platforms can seamlessly integrate with your existing frameworks, enhancing the course content and providing students with hands-on projects that mirror real-world applications.

I would love the opportunity to discuss how we can collaborate to amplify the impact of your new course and support your mission of shaping the future of AI education.

Looking forward to your thoughts!

Best regards,  
[Your Name]  
Lead Sales Representative  
CrewAI

---

### Email Draft 2: Highlighting Collaboration Opportunities

**Subject:** Let's Collaborate on Advancing AI Education!

Dear Andrew,

Congratulations on the remarkable success of the "Agentic AI" course! The hands-on approach you’ve incorporated is a game-changer for learners eager to dive into AI development.

At CrewAI, we recognize the challenges of scaling educational offerings while maintaining high standards. That's why we specialize in providing tailored solutions that enhance learning experiences. We believe that by collaborating, we can co-create projects that not only enrich your course content but also engage learners in meaningful ways.

I’d be thrilled to explore potential partnership opportunities that align with your vision at DeepLearning.AI. Together, we can further elevate the educational journey for your students.

Warm regards,  
[Your Name]  
Lead Sales Representative  
CrewAI

---

### Email Draft 3: Offering Support for Scaling Operations

**Subject:** Supporting DeepLearning.AI's Growth Journey

Dear Andrew,

I am reaching out to extend my heartfelt congratulations on the recent launch of "Agentic AI." It’s exciting to see how you're shaping the landscape of AI education with such innovative offerings!

As you continue to scale your operations, I wanted to introduce CrewAI's advanced tools designed to streamline administrative tasks and boost student engagement. Our solutions can help you focus on what matters most—delivering exceptional educational experiences to your learners.

I would appreciate the chance to discuss how we can support DeepLearning.AI's growth and help you achieve your ambitious goals for the future.

Best wishes,  
[Your Name]  
Lead Sales Representative  
CrewAI

---

These drafts are designed to resonate with DeepLearning.AI’s mission and recent achievements while offering valuable partnership opportunities.